# Agents with LlamaIndex II - 

Sources [1](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/), [2](https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/agents/), [3](https://docs.llamaindex.ai/en/stable/examples/agent/custom_agent/), [4](https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent/), [5](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/agent_runner/), [6](https://medium.com/llamaindex-blog/data-agents-eed797d7972f), [LlamaHub Tools](https://llamahub.ai/?tab=tools)  
    
--- 
### Agents

An "agent" is an automated reasoning and decision engine. It takes in a user input/query and can make internal decisions for executing that query in order to return the correct result. The key agent components can include, but are not limited to:

+ Breaking down a complex question into smaller ones
+ Choosing an external Tool to use + coming up with parameters for calling the Tool
+ Planning out a set of tasks
+ Storing previously completed tasks in a memory module

LlamaIndex provides a comprehensive framework for building agents. This includes the following components:

+ Using agents with tools at a high-level to build agentic RAG and workflow automation use cases **(Previous Notebook)**
+ Low-level components for building and debugging agents
+ Core agent ingredients that can be used as standalone modules: query planning, tool use, and more **(Previous Notebook)**

### Tools

Having proper tool abstractions is at the core of building data agents. Defining a set of Tools is similar to defining any API interface, with the exception that these Tools are meant for agent rather than human use. We allow users to define both a Tool as well as a ToolSpec containing a series of functions under the hood.

When using an agent or LLM with function calling, the tool selected (and the arguments written for that tool) rely strongly on the tool name and description of the tools purpose and arguments. Spending time tuning these parameters can result in larges changes in how the LLM calls these tools.

A Tool implements a very generic interface - simply define __call__ and also return some basic metadata (name, description, function schema).
LlamaIndex offer a few different types of Tools:

+ FunctionTool: A function tool allows users to easily convert any user-defined function into a Tool. It can also auto-infer the function schema. **(Previous Notebook)**
+ QueryEngineTool: A tool that wraps an existing query engine. Note: since our agent abstractions inherit from BaseQueryEngine, these tools can also wrap other agents. **(Previous Notebook)**
+ Community contributed ToolSpecs that define one or more tools around a single service (like Gmail): see [LlamaHub](https://llamahub.ai/)
+ Utility tools for wrapping other tools to handle returning large amounts of data from a tool: see [OnDemandLoaderTool](https://docs.llamaindex.ai/en/stable/examples/tools/OnDemandLoaderTool/)

### Use Cases

The scope of possible use cases for agents is vast and ever-expanding. That said, here are some practical use cases that can deliver immediate value.

+ Agentic RAG: Build a context-augmented research assistant over your data that not only answers simple questions, but complex research tasks. (Previous Notebook)
+ SQL Agent: A subset of the above is a "text-to-SQL assistant" that can interact with a structured database. 
+ Workflow Assistant: Build an agent that can operate over common workflow tools like email, calendar.
+ Coding Assistant: Build an agent that can operate over code.

## Installing Packages

In [1]:
!pip install -qU pip
!pip install -qU google-genai
!pip install -qU python-dotenv
!pip install -qU pypdf
!pip install -qU docx2txt

In [2]:
!pip uninstall -y $(pip freeze | grep llama-index | cut -d= -f1)

Found existing installation: llama-index 0.14.12
Uninstalling llama-index-0.14.12:
  Successfully uninstalled llama-index-0.14.12
Found existing installation: llama-index-cli 0.5.3
Uninstalling llama-index-cli-0.5.3:
  Successfully uninstalled llama-index-cli-0.5.3
Found existing installation: llama-index-core 0.14.12
Uninstalling llama-index-core-0.14.12:
  Successfully uninstalled llama-index-core-0.14.12
Found existing installation: llama-index-embeddings-adapter 0.4.1
Uninstalling llama-index-embeddings-adapter-0.4.1:
  Successfully uninstalled llama-index-embeddings-adapter-0.4.1
Found existing installation: llama-index-embeddings-huggingface 0.6.1
Uninstalling llama-index-embeddings-huggingface-0.6.1:
  Successfully uninstalled llama-index-embeddings-huggingface-0.6.1
Found existing installation: llama-index-embeddings-instructor 0.4.1
Uninstalling llama-index-embeddings-instructor-0.4.1:
  Successfully uninstalled llama-index-embeddings-instructor-0.4.1
Found existing installati

In [8]:
#!pip install -qU llama-index 
#!pip install -qU llama-index-experimental
#!pip install -qU llama_hub
!pip install -qU llama-index-llms-ollama llama-index-embeddings-ollama llama_index.embeddings.huggingface llama_index.embeddings.openai
!pip install -qU --no-cache-dir llama-index-core llama-index-llms-openai llama-index-agent-openai llama-index-llms-openai llama-index-tools-code-interpreter llama-index-workflows llama-index-question-gen-openai llama-index-readers-web llama-index-tools-google openai pydantic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-web 0.4.5 requires llama-index-core<0.13,>=0.12.0, but you have llama-index-core 0.14.12 which is incompatible.
llama-index-tools-google 0.5.0 requires llama-index-core<0.13,>=0.12.0, but you have llama-index-core 0.14.12 which is incompatible.
llama-index-agent-openai 0.4.12 requires llama-index-core<0.13,>=0.12.41, but you have llama-index-core 0.14.12 which is incompatible.
llama-index-llms-openai 0.4.7 requires llama-index-core<0.13,>=0.12.41, but you have llama-index-core 0.14.12 which is incompatible.
llama-index-tools-code-interpreter 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.14.12 which is incompatible.
llama-index-question-gen-openai 0.3.1 requires llama-index-core<0.13,>=0.12.0, but you have llama-index-core 0.14.12 which is incompatible.
llama-i

In [11]:
try:
    import llama_index.core
    print(llama_index.core.__version__)
    from llama_index.agent.openai import OpenAIAgent
    from llama_index.tools.code_interpreter import CodeInterpreterToolSpec
    print(f"✅ Sucess! Core version: {llama_index.core.__version__}")
except ImportError as e:
    print(f"❌ Import error: {e}")
except Exception as e:
    print(f"⚠️ Another error: {e}")

0.12.52.post1
✅ Sucess! Core version: 0.12.52.post1


## Importing Packages

In [12]:
import os
import sys
import shutil
import glob
import logging
from pathlib import Path
from IPython.display import Image
import pydantic
from dotenv import load_dotenv
import openai
from google import genai

import warnings
warnings.filterwarnings('ignore')
import logging
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import pandas as pd

## Llamaindex LLMs
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import SummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.core.query_engine import PandasQueryEngine

## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node
from llama_index.core import Settings

## LlamaIndex Templates
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole

## LlamaIndex Agents
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.tools import QueryEngineTool

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

import nest_asyncio
nest_asyncio.apply()

load_dotenv("../../.env", override=True)

True

In [13]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
#openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

#### Defining Models

In [14]:
#model="gpt-4o"
model="gpt-4o-mini"
#model="gpt-5"

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.llm = OpenAI(temperature=0, 
                      model=model, 
                      #max_tokens=512
                      PRESENCE_PENALTY=-2,
                      TOP_P=1,
                     )

llm = Settings.llm

---

## 1- Function Tool  
A function tool is a simple wrapper around any existing function (both sync and async are supported!).  

In [15]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def subtract(x: int, y: int) -> int: 
    """Subtract the second number from the first number."""
    return (x - y)

def multiply(x: int, y: int) -> int: 
    """Multiply one number by the other number."""
    return (x * y)

def uppercase(x: str) -> str: 
    """Return the input string in uppercase."""
    return (x.upper())

def square(x: int) -> int: 
    """Return the square of the give number."""
    return (x + x)

add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
uppercase_tool = FunctionTool.from_defaults(fn=uppercase)
square_tool = FunctionTool.from_defaults(fn=square)

In [16]:
response = llm.predict_and_call(
    [add_tool, 
     subtract_tool, 
     multiply_tool, 
     uppercase_tool,
     square_tool,
    ], 
    "Tell me the output of 3 - 12 ", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: subtract with args: {"x": 3, "y": 12}
=== Function Output ===
-9
-9


### There is no criticism on the functioning of the tool; tools are called by similarity of query vs docstring

In [17]:
response = llm.predict_and_call(
    [add_tool, 
     subtract_tool, 
     multiply_tool, 
     uppercase_tool,
     square_tool,
    ], 
    "Tell me how much is three to the power of two", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: square with args: {"x": 3}
=== Function Output ===
6
6


---

## 2- [Coding Assistant and Interpreter](https://llamahub.ai/l/tools/llama-index-tools-code-interpreter?from=)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-code-interpreter/examples/code_interpreter.ipynb)

In [22]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
from llama_index.tools.code_interpreter import CodeInterpreterToolSpec

code_spec = CodeInterpreterToolSpec()
tools = code_spec.to_tool_list()

agent = FunctionAgent(
    tools=tools, 
    llm=llm, 
    verbose=True
)

In [29]:
!ls ../../Data/csv/california_housing_train.csv

../../Data/csv/california_housing_train.csv


In [31]:
prompt = """
The file is located at '../../Data/csv/california_housing_train.csv'. 
Run a SINGLE Python script that does the following:
1. Imports pandas and os.
2. Checks if the path exists using os.path.exists and PRINTS the result.
3. If it exists, loads it and PRINTS df.info() and df.describe().
4. Creates the scatter plot and PRINTS 'Plot saved successfully' after plt.savefig().

IMPORTANT: You must use PRINT statements for everything you want to see.
"""

# 5. Execution (Asynchronous)
# If you are in a Jupyter Notebook, use 'await'
response = await agent.run(user_msg=prompt)

print("\n--- FINAL AGENT RESPONSE ---")
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent

--- FINAL AGENT RESPONSE ---
The script executed successfully. Here are the results:

1. **Path Check**: The path exists: `True`
2. **DataFrame Info**:
   - The DataFrame has 17,000 entries and 9 columns.
   - All columns are of type `float64` and have no missing values.
3. **DataFrame Description**:
   - Summary statistics for each col

In [32]:
q = "What was the Python code used to show the columns?"
response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
To show the columns of a DataFrame in Python, particularly when using the pandas library, you can use the following code:

```python
import pandas as pd

# Sample DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': ['A', 'B', 'C'],
    'Column3': [True, False, True]
}

df = pd.DataFrame(data)

# Show the columns
print(df.columns)
```

This code creates a simple DataFrame and then prints the names of its columns. If you have a specific DataFrame in mind or need further assistance, please let me know!


In [37]:
q = "In which country are these latitudes and longitudes in the Dataframe read from '../../Data/csv/california_housing_train.csv'?"
response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The DataFrame contains latitude and longitude data. Here are the first few rows:

- Longitude: -114.31, Latitude: 34.19
- Longitude: -114.47, Latitude: 34.40
- Longitude: -114.56, Latitude: 33.69
- Longitude: -114.57, Latitude: 33.64
- Longitude: -114.57, Latitude: 33.57

These coordinates are located in the United States, specifically i

In [38]:
q = "In which country are these latitudes and longitudes in the Dataframe read from '../../Data/csv/california_housing_train.csv'?"

response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The DataFrame contains latitude and longitude data. Here are the first few rows:

- Longitude: -114.31, Latitude: 34.19
- Longitude: -114.47, Latitude: 34.40
- Longitude: -114.56, Latitude: 33.69
- Longitude: -114.57, Latitude: 33.64
- Longitude: -114.57, Latitude: 33.57

These coordinates are located in the United States, specifically i

In [40]:
q = "Can you make a graph of the median income per total rooms in the Dataframe read from '../../Data/csv/california_housing_train.csv'?"

response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The graph of median income per total rooms has been created successfully. I saved the graph as an image file named `median_income_per_rooms.png`. You can download it using the link below:

[Download median_income_per_rooms.png](sandbox:/median_income_per_rooms.png)


---

## 3- [SQL Agent](https://llamahub.ai/l/tools/llama-index-tools-database?from=)  
[GitHub](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-database/examples/database.ipynb), [article](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo/)

In [42]:
%pip install -qU llama-index-tools-database
%pip install -q sqlalchemy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.tools.database import DatabaseToolSpec
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    insert,
    text,
)

In [44]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [45]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

#### Define SQL Database   

We first define our SQLDatabase abstraction (a light wrapper around SQLAlchemy).

In [46]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

#### We add some testing data to our SQL database

In [47]:
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States",},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [48]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


#### Query Index  
We first show how we can execute a raw SQL query, which directly executes over the table.  

In [49]:
with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


#### Part 1: Text-to-SQL Query Engine¶

Once we have constructed our SQL database, we can use the NLSQLTableQueryEngine to construct natural language queries that are synthesized into SQL queries.

Note that we need to specify the tables we want to use with this query engine. If we don't the query engine will pull all the schema context, which could overflow the context window of the LLM.

This query engine should be used in any case where you can specify the tables you want to query over beforehand, or the total size of all the table schema plus the rest of the prompt fits your context window.

In [50]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [51]:
from IPython.display import Markdown
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

#### Part 2: Query-Time Retrieval of Tables for Text-to-SQL

If we don't know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the SQLTableNodeMapping object, which takes in a SQLDatabase and produces a Node object for each SQLTableSchema object passed into the ObjectIndex constructor.  

In [52]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
#from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

#### Now we can take our SQLTableRetrieverQueryEngine and query it for our response.

In [53]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

In [54]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Tokyo', 13960000)]

#### You could also add additional context information for each table schema you define.

In [55]:
# manually set context text
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

#### Part 3: Text-to-SQL Retriever

So far our text-to-SQL capability is packaged in a query engine and consists of both retrieval and synthesis.

You can use the SQL retriever on its own. We show you some different parameters you can try, and also show how to plug it into our RetrieverQueryEngine to get roughly the same results.


In [56]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [57]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [58]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 3f237861-a6e8-4bdb-99f5-9ff5c870b0aa<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [59]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [60]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [61]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** b24a1777-72f3-40b6-803e-a9c2e34f0342<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** 77d266e5-f342-4992-8a99-b0b75ea2b3d0<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** 35be7e4b-db9a-48dd-95b1-aaf5e66d5133<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** 458df849-f99e-40d1-99ff-1acb009a6ca3<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

#### Plug into our RetrieverQueryEngine

We compose our SQL Retriever with our standard RetrieverQueryEngine to synthesize a response. The result is roughly similar to our packaged Text-to-SQL query engines.

In [ ]:
#from llama_index.core.agent.workflow import FunctionAgent
#from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [63]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [64]:
print(str(response))

1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000


#### Using Database Tool

This tool connects to a database (using SQLAlchemy under the hood) and allows an Agent to query the database and get information about the tables.

In [76]:
db_tools = DatabaseToolSpec(sql_database)
tools = db_tools.to_tool_list()
agent = FunctionAgent(
    tools=tools, 
    llm=llm, 
    verbose=True
)

prompt = "What tables does this database contain?"
response = await agent.run(user_msg=prompt)

print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The database contains the table named **city_stats**.


In [80]:
q = "Describe the table *city_stats*"
response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The table `city_stats` has the following structure:

- **city_name**: A VARCHAR field with a maximum length of 16 characters, which cannot be null. This serves as the primary key for the table.
- **population**: An INTEGER field that represents the population of the city.
- **country**: A VARCHAR field with a maximum length of 16 charact

In [81]:
q = "Retrieve the first row of that table *city_stats*"
response = await agent.run(user_msg=q)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

---

## 4- Community Tools

### A) [Wikipedia Tool](https://llamahub.ai/l/tools/llama-index-tools-wikipedia?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-wikipedia/examples/wikipedia.ipynb)

In [82]:
%pip install -q llama-index-tools-wikipedia

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#from llama_index.core.agent.workflow import FunctionAgent
#from llama_index.llms.openai import OpenAI
from llama_index.tools.wikipedia import WikipediaToolSpec

tool_spec = WikipediaToolSpec()
tools = tool_spec.to_tool_list()

agent = FunctionAgent(
    tools=tools, 
    llm=llm, 
    verbose=True
)

response = await agent.run(user_msg="Who is Ben Affleck's spouse?")
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
Ben Affleck is currently married to Jennifer Lopez. They began dating again in April 2021, after previously being in a relationship from 2002 to 2004. They announced their second engagement in April 2022 and were married in a Las Vegas ceremony on July 16, 2022.


### B) [Arxiv Tool](https://llamahub.ai/l/tools/llama-index-tools-arxiv?from=tools)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-arxiv)  

In [85]:
%pip install -q llama-index-tools-arxiv

Note: you may need to restart the kernel to use updated packages.


In [86]:
#from llama_index.core.agent.workflow import FunctionAgent
#from llama_index.llms.openai import OpenAI
from llama_index.tools.arxiv.base import ArxivToolSpec

arxiv_tool = ArxivToolSpec()
tools = arxiv_tool.to_tool_list()

agent = FunctionAgent(
    tools=tools, 
    llm=llm, 
    verbose=True
)

prompt = "What are the newest discoveries on Prompt Engineering? Summarize the top 3 most recent papers."
response = await agent.run(user_msg=prompt)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_ag

### C) [DuckDuckGoSearch Tool](https://llamahub.ai/l/tools/llama-index-tools-duckduckgo?from=tools)  
[Github](https://github.com/run-llama/llama_index/blob/main/llama-index-integrations/tools/llama-index-tools-duckduckgo/examples/duckduckgo_search.ipynb)  

In [87]:
%pip install -q llama-index-tools-duckduckgo

Note: you may need to restart the kernel to use updated packages.


In [88]:
#from llama_index.core.agent.workflow import FunctionAgent
#from llama_index.llms.openai import OpenAI
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()
tools = tool_spec.to_tool_list()

agent = FunctionAgent(
    tools=tools,
    llm=llm,
    verbose=True
)
prompt = "what are the latest developments in machine learning"
response = await agent.run(user_msg=prompt)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

### D) [Yahoo Finance Tool](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/tools/)  
[Github](https://github.com/run-llama/llama_index/tree/main/llama-index-integrations/tools/llama-index-tools-yahoo-finance)  

In [90]:
%pip install -q llama-index-tools-yahoo-finance

Note: you may need to restart the kernel to use updated packages.


In [91]:
#from llama_index.core.agent.workflow import FunctionAgent
#from llama_index.llms.openai import OpenAI
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

tool_spec = YahooFinanceToolSpec()
tools = tool_spec.to_tool_list()

agent = FunctionAgent(
    tools=tools,
    llm=llm,
    verbose=True
)
prompt = "What is the price of Apple stock?"
response = await agent.run(user_msg=prompt)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
The current price of Apple Inc. (AAPL) stock is $255.53.


In [94]:
q = "What is the latest news about Apple Inc. (AAPL)?"
response = await agent.run(user_msg=q, max_iterations=20)
print(response)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag